<style></style>
<h1>Cyfrowe przetwarzanie sygnałów i obrazów</h1>
<h2>Przetwarzanie i analiza sygnału EKG</h2>
<b>Autorki:</b> Barbara Parzonka 263955, Joanna Zoglowek 264452
<br><br>
<b>Ćwiczenie 1.</b>
Napisz skrypt w Pythonie/Matlabie umożliwiający wczytywanie i wizualizację badanych sygnałów. Program powinien umożliwiać obserwowanie wycinka sygnału dla zadanego przedziału czasowego, skalowanie osi wykresów i ich opis oraz
zapis dowolnego wycinka sygnału do pliku o podanej nazwie.
<br><br>
<b>Opis realicaji zadania</b> <br>
Wywołanie programu poprez <i>Run>Run All Cells</i> wiąże się z wybraniem domyślnych ustawień dotyczących wczytywanego z pliku tekstowego sygnału (sygnał wczytany z pliku <i>ekg100.txt</i>, częstotliwość próbkowania 360 Hz). Jednym z ustawień dotyczących sygnału jest pole wyboru tego, czy pierwsza kolumna pliku tekstowego ma być traktowana jako czas poboru sygnału. 
<br>
Program jest podzielony na trzy komórki - wywołanie pierwszej z nich tworzy i incjalizuje część zmiennych potrzebnych w programie oraz odpowiada za interfejs umożliwiający użytkownikowi ustawienie wspomianych wyżej właściwości sygnnału (nazwa pliku, częstotliwość, stwierdzenie, czy pierwsza kolumna to czas). <br>
Wywołanie drugiej komórki powinno odbyć się po uprzednim wywołaniu pierwszej komórki. Ta część jest odpowiedzialna za odczytanie odpowiedniego pliku tesktowego.
Zadaniem trzeciej części jest narysowanie wykresu i umozliwienie interakcji z wykresem - w tym zmianę wyświetlanego przedziału i zapis sygnału do pliku o zadanej nazwie.

In [1]:
%run methods.ipynb
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import asyncio
import sys
xarray=[]
valuesArray=[]
numberofCases=1
datapath='../data/'
bottomLimit=0
upperLimit=len(xarray)


fileOption=widgets.RadioButtons(
    options=['ekg100.txt', 'ekg1.txt', 'ekg_noise.txt'],
    description='Wybierz plik z danymi: \n',
    disabled=False
)

freq=widgets.IntText(
    value=360,
    min=1,
    description='Częstotliwość próbkowania (Hz):',
    disabled=False,
)
freq.style.description_width = 'auto'

timeFirst=widgets.Checkbox(
    value=False,
    description='W pierwszej kolumnie znajduje sie czas',
    disabled=False
)
timeFirst.style.description_width = 'auto'

grid = widgets.GridspecLayout(3,3)
grid[:, 0] = fileOption
grid[1, 1:] = freq
grid[2, 1:] = timeFirst
grid[:, 0].layout.height = 'auto'

display(grid)


GridspecLayout(children=(RadioButtons(description='Wybierz plik z danymi: \n', layout=Layout(grid_area='widget…

In [3]:
filepath=datapath+fileOption.value
frequency=int(freq.value)
hasFirstTime=timeFirst.value
out = widgets.Output(layout={'border': '1px solid black'})

def plotEKG(domain):
    global bottomLimit, upperLimit, numberOfCases
    if domain[1]>0:
        bottomLimit=int(domain[0]*frequency)
        upperLimit=int(domain[1]*frequency)
    else:
        bottomLimit=0
        upperLimit=len(xarray)
    plt.figure(figsize=(15, 3 * numberOfCases))  # Ustawienie większego rozmiaru figury
    plt.subplots_adjust(hspace=1.0)  # Większe odstępy między wykresami
    
    for i in range(numberOfCases):
        plt.subplot(numberOfCases, 1, i+1)
        plt.plot(xarray[bottomLimit:upperLimit], valuesArray[i][bottomLimit:upperLimit], label=f'Sygnał {i+1}')
        plt.xlabel('Czas [s]')
        if i== int(numberOfCases/2):
            plt.ylabel('Amplituda [mV]')
        plt.legend(loc='upper right')
        plt.grid(True)
    plt.tight_layout()  # Automatyczna regulacja odstępów
    plt.show()


def zadanie1(frequency=frequency, filename=filepath):
    if frequency<=0:
        with out:
            print('Podano nieodpowiednia czestotliwosc')
    else:
        if readFile(filepath, frequency, hasFirstTime):
            return True
        else:
            exit()
    
def save_values_to_file(file_path):
    global bottomLimit,upperLimit, numberOfCases
    try:
        with open(file_path, "w") as file:
            for i in range (bottomLimit,upperLimit):
                pom=''
                for j in range(numberOfCases):
                    pom+=str(float(valuesArray[j][i]))+" "
                    file.write(pom+"\n")
        with out:
            print("Zapisano w pliku o ścieżce:", file_path)
    except Exception as e:
        with out:
            print("Podczas zapisu doszlo do wyjatku: ", e)
allSet=zadanie1()


In [4]:
domain_slider=widgets.FloatRangeSlider(
    value=[0,len(xarray)* 1 / frequency],
    min=0, 
    max=len(xarray)* 1 / frequency, 
    step=0.1,
    desciption='Przedzial czasowy',
    continuous_update=False,
    layout=Layout(width='80%')
)


file_name_input = widgets.Text(
    placeholder='Podaj nazwę pliku',
    description='Nazwa pliku, do którego ma zostać zapisana wartość:',
    disabled=False
)

def on_save_button_clicked(b):
    file_name = file_name_input.value.strip()
    if file_name:
        file_path =datapath+file_name + ".txt"  # Assuming you want to save as text file
        save_values_to_file(file_path)
    else:
        with out:
            print('Nazwa pliku jest nieodpowiednia')
    plt.xlim(1,3)

save_button = widgets.Button(
    description='Zapisz do pliku'
)
save_button.on_click(on_save_button_clicked)

display(out)

if allSet:
    interact(plotEKG,domain=domain_slider)
    display(file_name_input)
    display(save_button) 


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

interactive(children=(FloatRangeSlider(value=(0.0, 685.3444444444444), continuous_update=False, description='d…

Text(value='', description='Nazwa pliku, do którego ma zostać zapisana wartość:', placeholder='Podaj nazwę pli…

Button(description='Zapisz do pliku', style=ButtonStyle())